# Drugi etap projektu
Julia Jodczyk

Filip Pawłowski 
### Polecenie:
“Jakiś czas temu wprowadziliśmy konta premium, które uwalniają użytkowników od słuchania reklam. Nie są one jednak jeszcze zbyt popularne – czy możemy się dowiedzieć, które osoby są bardziej skłonne do zakupu takiego konta?”

In [ ]:
import pickle
import requests
import json
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from microservice.load_data import Preprocessor 
from microservice.files_utils import randomly_split_group

## Modele

Stworzyliśmy modele klasyfikacji binarnej, które dzielą użytkowników na grupy: `kupi premium` i `nie kupi premium`. 

### Model Bazowy

Jako model bazowy, najprostszy z możliwych dla danego zadania uznajemy model naiwny, który zawsze klasyfikuje użytkowników do grupy `kupi premium`: 

In [ ]:
class NaiveModel:
    def predict(self, input_data):
        return [1]
    
base_model = NaiveModel()

with open('./microservice/saved_models/base_model.sav', 'wb') as f:
    pickle.dump(base_model, f)

### Model docelowy

Jako model docelowy, po analizie z pierwszego etapu, wybraliśmy model KNN z następującymi cechami (per użytkownik):
- miasto 
- stosunek czasu reklam do całego czasu, jaki użytkownik spędził korzystając z serwisu
- stosunkowy udział każdego typu zdarzenia (event_type) we wszystkich zdarzeniach sesji
- stosunek ilości reklam po utworach ulubionego gatunku
- ulubione gatunki użytkownika

Implementacja ekstrakcji powyższych cech została umieszczona w pliku `load_data.py`. Cechy nieliczbowe - miasto oraz ulubione gatunki zostały zakodowane sposobem one hot encoding.

In [ ]:
with open("./microservice/data/users.json", "r") as f:
        group = json.load(f)
train_users, test_users = randomly_split_group(group)
with open("./microservice/data/train_users.json", "w") as f:
    json.dump(train_users, f)

In [ ]:
target_model = KNeighborsClassifier()
# load data:
# use file train_users instead of users
X_train, y_train = Preprocessor.run()
# split data:
target_model.fit(X_train, y_train)

with open('./microservice/saved_models/KNN_model.sav', 'wb') as f:
    pickle.dump(target_model, f)

### Porównanie wyników

Wyniki predykcji zbierzemy za pomocą zaimplementowanego mikroserwisu (szczegóły implementacji i API niżej). 

(Przed uruchomieniem kodu z poniższej komórki należy uruchomić mikroserwis komendą `python3 /microservice/microservice.py`) 

In [ ]:
base_url = "http://127.0.0.1:8000"

base_model_test, target_model_test = randomly_split_group(test_users)
for user in target_model_test:
    requests.post(f"{base_url}/predict-with/KNN", params={"test": "True"}, json=user)
    actual_body = {
        "user_id": user["user_id"],
        "actual": user["premium_user"]
    }
    requests.post(f"{base_url}/submit-actual", json=actual_body)
for user in base_model_test:
    requests.post(f"{base_url}/predict-with/base", params={"test": "True"}, json=user)
    actual_body = {
        "user_id": user["user_id"],
        "actual": user["premium_user"]
    }
    requests.post(f"{base_url}/submit-actual", json=actual_body)


In [ ]:
response = requests.get(f"{base_url}/test_ab_results")
